In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

**1. 设置输入和输出节点的个数,配置神经网络的参数**

In [2]:
INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

BATCH_SIZE = 100

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 10000
MOVING_AVERAGE_DECAY = 0.99

In [3]:
DATA_SRC = "../../datasets/mnist/"

**2. 定义辅助函数来计算前向传播结果，使用ReLU做为激活函数。**

In [4]:
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 计算参数平均值,可以使用滑动平均模型
        layer1 = tf.nn.relu(
            tf.matmul(input_tensor, avg_class.average(weights1))
            + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)

**3. 定义训练过程。**

In [5]:
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 生成隐藏层的参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    # 生成输出层的参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev = 0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算在当前参数下的神经网络前向传播的结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量
    global_step = tf.Variable(0, trainable=False)
    
    # 给定滑动平均衰减率
    variable_averages = tf.train.ExponentialMovingAverage(
        MOVING_AVERAGE_DECAY, global_step)
    
    # 在所有神经网络参数的变量上使用滑动平均
    variables_averages_op = variable_averages.apply(
        tf.trainable_variables())
    
    #计算使用了滑动平均值后的前向传播结果
    average_y = inference(
        x, variable_averages, weights1, biases1, weights2, biases2)
    
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失
    regularization = regularizer(weights1) + regularizer(weights2)
    
    # 计算总损失
    loss = cross_entropy_mean + regularization
    
    # 设置指数衰减学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE, # 基础学习率
        global_step,        # 当前迭代的轮数
        mnist.train.num_examples / BATCH_SIZE,  # 训练总次数
        LEARNING_RATE_DECAY, # 学习率的衰减速度
        staircase=True) 
    
    # 优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate)\
                   .minimize(loss, global_step=global_step)

    # 反向传播更新参数和更新每一个参数的滑动平均值
    #train_op = tf.group(train_step, varoan;es_averages_op)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')

    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # 初始化会话,并开始训练过程
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        # 准备验证数据
        validate_feed = {x: mnist.validation.images,
                         y_: mnist.validation.labels}

        # 准备测试数据
        test_feed = {x: mnist.test.images,
                     y_: mnist.test.labels}

        # 迭代地训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证集上的测试结果
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step(s), validation accuracy "
                      "using average model is %g" % (i, validate_acc))

            # 产生这一轮使用的一个batch的训练数据,并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x:xs, y_:ys})


        # 训练结束后,在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training steps, test accuracy using average "
              "model is %g" % (TRAINING_STEPS, test_acc))


**4. 主程序入口，这里设定模型训练次数为10000次。**

In [6]:
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets(DATA_SRC, one_hot = True)
    train(mnist)
    
if __name__ == '__main__':
    main()


Extracting ../../datasets/mnist/train-images-idx3-ubyte.gz
Extracting ../../datasets/mnist/train-labels-idx1-ubyte.gz
Extracting ../../datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting ../../datasets/mnist/t10k-labels-idx1-ubyte.gz


InternalError: Blas GEMM launch failed : a.shape=(5000, 784), b.shape=(784, 500), m=5000, n=500, k=784
	 [[Node: MatMul_2 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_x-input_0_0/_7, Variable/ExponentialMovingAverage/read)]]
	 [[Node: Mean_1/_9 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_33_Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'MatMul_2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packa